In [1]:
import networkx as nx
import matplotlib.pyplot as plt
import os
from BNS_JT import trans
from BNS_JT import variable, cpm
import numpy as np

import BNS_JT
from BNS_JT import gen_bnb
import copy

HOME = os.getcwd()


In [2]:
import importlib
importlib.reload(gen_bnb)

<module 'BNS_JT.gen_bnb' from 'c:\\Users\\jb622s\\git\\BNS-JT\\BNS_JT\\gen_bnb.py'>

# Results of general B&B algorithm to reliability analysis

Complete branch and bound, deterministic inference, and indepdent components

# Problem

## Network

In [3]:
# Network
node_coords = {'n1': (-2, 3),
               'n2': (-2, -3),
               'n3': (2, -2),
               'n4': (1, 1),
               'n5': (0, 0)}

arcs = {'e1': ['n1', 'n2'],
	'e2': ['n1', 'n5'],
	'e3': ['n2', 'n5'],
	'e4': ['n3', 'n4'],
	'e5': ['n3', 'n5'],
	'e6': ['n4', 'n5']}

arcs_avg_kmh = {'e1': 40,
                'e2': 40,
                'e3': 40,
                'e4': 30,
                'e5': 30,
                'e6': 20}

arc_lens_km = trans.get_arcs_length(arcs, node_coords)
arc_times_h = {k: v/arcs_avg_kmh[k] for k, v in arc_lens_km.items()}

comps_name = [k for k in arcs] # *TODO* this is not necessary if branch's down and up are defined by dictionary (instead of list)


od_pair = ('n1', 'n3')


# Component events
no_arc_st = 3 # number of component states 
delay_rat = [10, 2, 1] # delay in travel time given each component state (ratio)
varis = {}
for k, v in arcs.items():
    varis[k] = variable.Variable( name=k, B = np.eye( no_arc_st ), values = [arc_times_h[k]*np.float64(x) for x in delay_rat])


# plot graph
G = nx.Graph()
for k, x in arcs.items():
    G.add_edge(x[0], x[1], time=arc_times_h[k], label=k)

for k, v in node_coords.items():
    G.add_node(k, pos=v)

pos = nx.get_node_attributes(G, 'pos')
edge_labels = nx.get_edge_attributes(G, 'label')

fig = plt.figure()
ax = fig.add_subplot()
nx.draw(G, pos, with_labels=True, ax=ax)
nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels, ax=ax)
fig.savefig(os.path.join(HOME, 'graph.png'), dpi=200)

## Events: Independent component events

In [4]:
cpms = {}

# Component events
for k in arcs:
    cpms[k] = cpm.Cpm( variables=[ varis[k] ], no_child = 1, C = np.array([1,2,3])-1, p = [0.1, 0.2, 0.7] ) 

# Damage observation
C_o = np.array([[1,1], [2,1], [3,1], [1,2], [2,2], [3,2], [1,3], [2,3], [3,3]])-1
p_o = np.array([0.95, 0.04, 0.01, 0.3, 0.5, 0.2, 0.01, 0.19, 0.8]).T
for (i,k) in enumerate(arcs):
    name = 'o' + str(i+1)

    varis[name] = variable.Variable( name=name, B = np.eye( no_arc_st ), values = [0,1,2] ) # observation that e_i = 0, 1, or 2 ** TO DISCUSS: probably values in dictionary..?
    cpms[name] = cpm.Cpm( variables=[ varis[name], varis[k] ], no_child = 1, C = C_o, p = p_o )

print(cpms['e1'])
print(cpms['o1'])

Cpm(variables=['e1'], no_child=1, C=[[0]
 [1]
 [2]], p=[[0.1]
 [0.2]
 [0.7]]
Cpm(variables=['o1', 'e1'], no_child=1, C=[[0 0]
 [1 0]
 [2 0]
 [0 1]
 [1 1]
 [2 1]
 [0 2]
 [1 2]
 [2 2]], p=[[0.95]
 [0.04]
 [0.01]
 [0.3 ]
 [0.5 ]
 [0.2 ]
 [0.01]
 [0.19]
 [0.8 ]]


# System event quantification by general B&B algorithm

## System function

In [5]:
def get_time_and_path( comps_st, od_pair, arcs, vari ):
    G = nx.Graph()
    for k, x in arcs.items():
        c_st = comps_st[k]
        G.add_edge(x[0], x[1], time=vari[k].values[c_st])

    path = nx.shortest_path( G, source = od_pair[0], target = od_pair[1], weight = 'time' )
    time = nx.shortest_path_length( G, source = od_pair[0], target = od_pair[1], weight = 'time' )

    return time, path


def sf_min_path( comps_st, od_pair, arcs, vari, thres, sys_val_itc ):

    sys_val, path = get_time_and_path( comps_st, od_pair, arcs, vari )

    if sys_val > thres*sys_val_itc:
        sys_st = 'fail'
    else:
        sys_st = 'surv'

    if sys_st == 'surv': # in this case we know how to find out minimally required component state
        min_comps_st = {}
        for i in range(len(path)-1):
            nodes_i = [path[i], path[i+1]]
            nodes_i_rev = [path[i+1], path[i]] # reversed pair (arcs are bi-directional)
            arc_i = next((k for k, v in arcs.items() if v == nodes_i or v == nodes_i_rev), None)
            min_comps_st[arc_i] = comps_st[arc_i]
    
    else: # sys_st == 'fail'
        min_comps_st = None

    return sys_val, sys_st, min_comps_st

In [6]:
# Intact state of component vector
comps_st_itc = {k: len(varis[k].B[0])-1 for k in arcs} # intact state (i.e. the highest state)
sys_val_itc, path_itc = get_time_and_path( comps_st_itc, od_pair, arcs, varis )

# defines the system failure event
thres = 2 

# Given a system function, i.e. sf_min_path, it should be represented by a function that only has "comps_st" as input.
sys_fun = lambda comps_st : sf_min_path( comps_st, od_pair, arcs, varis, thres, sys_val_itc ) # TODO: branch needs to have states defined in dictionary instead of list.

## Branch and Bound: complete results, i.e. no unknown branches

In [7]:
import importlib
importlib.reload(gen_bnb)

<module 'BNS_JT.gen_bnb' from 'c:\\Users\\jb622s\\git\\BNS-JT\\BNS_JT\\gen_bnb.py'>

In [8]:
varis_comp = {}
for k in arcs:
    varis_comp[k] = copy.deepcopy( varis[k] )
    

no_sf, rules, rules_st, brs, sys_res = gen_bnb.do_gen_bnb( sys_fun, varis_comp, max_br=100 ) # Complete 

[Iteration 1]..
The # of found non-dominated rules: 0
The # of branches: 0
---
[Iteration 2]..
The # of found non-dominated rules: 1
The # of branches: 1
---
[Iteration 3]..
The # of found non-dominated rules: 2
The # of branches: 1
---
[Iteration 4]..
The # of found non-dominated rules: 3
The # of branches: 1
---
[Iteration 5]..
The # of found non-dominated rules: 3
The # of branches: 1
---
[Iteration 6]..
The # of found non-dominated rules: 4
The # of branches: 1
---
[Iteration 7]..
The # of found non-dominated rules: 4
The # of branches: 1
---
[Iteration 8]..
The # of found non-dominated rules: 5
The # of branches: 1
---
[Iteration 9]..
The # of found non-dominated rules: 5
The # of branches: 2
---
[Iteration 10]..
The # of found non-dominated rules: 6
The # of branches: 2
---
[Iteration 11]..
The # of found non-dominated rules: 6
The # of branches: 4
---
[Iteration 12]..
The # of found non-dominated rules: 7
The # of branches: 4
---
[Iteration 13]..
The # of found non-dominated rul

In [9]:
for b in brs:
    print(b)

Branch(down=[0, 0, 0, 0, 0, 0], up=[2, 0, 1, 2, 2, 2], is_complete=True, down_state=fail, up_state=fail, down_val=None, up_val=None)
Branch(down=[0, 0, 2, 0, 0, 0], up=[2, 0, 2, 2, 1, 2], is_complete=True, down_state=fail, up_state=fail, down_val=None, up_val=None)
Branch(down=[0, 0, 2, 0, 2, 0], up=[1, 0, 2, 2, 2, 2], is_complete=True, down_state=fail, up_state=fail, down_val=None, up_val=None)
Branch(down=[2, 0, 2, 0, 2, 0], up=[2, 0, 2, 2, 2, 2], is_complete=True, down_state=surv, up_state=surv, down_val=None, up_val=None)
Branch(down=[0, 1, 0, 0, 0, 0], up=[2, 2, 2, 1, 0, 2], is_complete=True, down_state=fail, up_state=fail, down_val=None, up_val=None)
Branch(down=[0, 1, 0, 2, 0, 0], up=[2, 2, 2, 2, 0, 0], is_complete=True, down_state=fail, up_state=fail, down_val=None, up_val=None)
Branch(down=[0, 1, 0, 2, 0, 1], up=[2, 1, 2, 2, 0, 1], is_complete=True, down_state=fail, up_state=fail, down_val=None, up_val=None)
Branch(down=[0, 2, 0, 2, 0, 1], up=[2, 2, 2, 2, 0, 1], is_complete=Tr

# System Reliability Analysis

## Quantification of system events

### Function: Finding a composite state corresponding to a given list of states

If there is no corresponding compatible state, it creates a new composite state.

This function needs to be in 'Variable.py'.

In [10]:
def get_compsite_state( var1, st_list ):
    # Input: var1-one Variable object, st_list: list of states (starting from zero) 
    # TODO: states start from 0 in Cpm and from 1 in B&B -- will be fixed later so that all start from 0

    n_st = len( var1.B[0] )
    b1 = np.zeros( (n_st,), dtype = int )
    for s in st_list:
        b1[s] = 1

    comp_st_list = np.where((var1.B == b1).all(axis=1))[0]
    if len(comp_st_list) > 0:
        comp_st = comp_st_list[0]
    else:
        B_old = var1.B
        B_new = np.vstack( (B_old, b1) )
        var1.B = B_new 
        comp_st = len(var1.B)-1 # zero-based index
    
    return var1, comp_st   


In [11]:
# ex 1: there is no corresponding composite state for state 1 and 2. So it creates one.
var1 = copy.deepcopy( varis['e1'] )
print(var1)
var1_new, comp_st = get_compsite_state( var1, [1,2] )
print(comp_st)
print(var1_new)
print("")

# ex 2: now there is such composite state, and returns the corresponding one.
var1_new2, comp_st = get_compsite_state( var1_new, [1,2] )
print(comp_st, var1_new2)

'Variable(name=e1, B=[[1 0 0]\n [0 1 0]\n [0 0 1]], values=[1.5, 0.3, 0.15])'
3
'Variable(name=e1, B=[[1 0 0]\n [0 1 0]\n [0 0 1]\n [0 1 1]], values=[1.5, 0.3, 0.15])'

3 'Variable(name=e1, B=[[1 0 0]\n [0 1 0]\n [0 0 1]\n [0 1 1]], values=[1.5, 0.3, 0.15])'


### Function: From branch to C matrix of system event

In [12]:
def get_c1_from_br1( br1, varis, st_br_to_cs ):
    # br1: a single branch
    # varis: a dictionary of variables
    # st_br_to_cs: a dictionary that maps state in br to state in C matrix of a system event

    comps_name = br1.names

    cs_nvar = 1 + len(comps_name) # (system, compponents)
    c1 = np.zeros( cs_nvar, int )

    if br1.is_complete == True:
        c1[0] = st_br_to_cs[ br1.down_state ]
    else:
        c1[0] = st_br_to_cs[ 'unk' ]

    for (i,x) in enumerate( comps_name ):
        x_d = br1.down[i]
        x_u = br1.up[i]

        if x_u > x_d:
            varis[x], x_st = get_compsite_state( varis[x], list((range(x_d,x_u+1))  ))
        else:
            x_st = x_u

        c1[i+1] = x_st

    return varis, c1

In [13]:
st_br_to_cs = {'fail': 0, 'surv': 1, 'unk': 2}

# ex1
br1 = copy.deepcopy( brs[0] )
varis, c1 = get_c1_from_br1( br1, varis, st_br_to_cs )
print(br1)
print(c1)
print(varis['e1'])
print(varis['e2'])
print(varis['e5'])
print("")

# ex2
br1 = copy.deepcopy( brs[1] )
varis, c1 = get_c1_from_br1( br1, varis, st_br_to_cs )
print(br1)
print(c1)
print(varis['e5'])
print(varis['e2'])


Branch(down=[0, 0, 0, 0, 0, 0], up=[2, 0, 1, 2, 2, 2], is_complete=True, down_state=fail, up_state=fail, down_val=None, up_val=None)
[0 3 0 3 3 3 3]
'Variable(name=e1, B=[[1 0 0]\n [0 1 0]\n [0 0 1]\n [1 1 1]], values=[1.5, 0.3, 0.15])'
'Variable(name=e2, B=[[1 0 0]\n [0 1 0]\n [0 0 1]], values=[0.9013878188659973, 0.18027756377319945, 0.09013878188659973])'
'Variable(name=e5, B=[[1 0 0]\n [0 1 0]\n [0 0 1]\n [1 1 1]], values=[0.9428090415820635, 0.1885618083164127, 0.09428090415820635])'

Branch(down=[0, 0, 2, 0, 0, 0], up=[2, 0, 2, 2, 1, 2], is_complete=True, down_state=fail, up_state=fail, down_val=None, up_val=None)
[0 3 0 2 3 4 3]
'Variable(name=e5, B=[[1 0 0]\n [0 1 0]\n [0 0 1]\n [1 1 1]\n [1 1 0]], values=[0.9428090415820635, 0.1885618083164127, 0.09428090415820635])'
'Variable(name=e2, B=[[1 0 0]\n [0 1 0]\n [0 0 1]], values=[0.9013878188659973, 0.18027756377319945, 0.09013878188659973])'


In [14]:
def get_Csys_from_brs( brs, varis, st_br_to_cs ):

    comps_name = brs[0].names

    cs_nvar = 1 + len(comps_name) # (system, compponents)

    Csys = np.empty( (0,cs_nvar), int )
    
    for br1 in brs:
        varis, c1 = get_c1_from_br1( br1, varis, st_br_to_cs )
        Csys = np.vstack( (Csys, c1) )

    return Csys, varis


In [15]:
Csys, varis = get_Csys_from_brs( brs, varis, st_br_to_cs )

print(Csys)
for v in varis:
    print( varis[v] )

[[0 3 0 3 3 3 3]
 [0 3 0 2 3 4 3]
 [0 4 0 2 3 2 3]
 [1 2 0 2 3 2 3]
 [0 3 3 4 4 0 3]
 [0 3 3 4 2 0 0]
 [0 3 1 4 2 0 1]
 [1 3 2 4 2 0 1]
 [1 3 3 4 2 0 2]
 [1 3 3 4 3 5 3]]
'Variable(name=e1, B=[[1 0 0]\n [0 1 0]\n [0 0 1]\n [1 1 1]\n [1 1 0]], values=[1.5, 0.3, 0.15])'
'Variable(name=e2, B=[[1 0 0]\n [0 1 0]\n [0 0 1]\n [0 1 1]], values=[0.9013878188659973, 0.18027756377319945, 0.09013878188659973])'
'Variable(name=e3, B=[[1 0 0]\n [0 1 0]\n [0 0 1]\n [1 1 0]\n [1 1 1]], values=[0.9013878188659973, 0.18027756377319945, 0.09013878188659973])'
'Variable(name=e4, B=[[1 0 0]\n [0 1 0]\n [0 0 1]\n [1 1 1]\n [1 1 0]], values=[1.0540925533894598, 0.21081851067789198, 0.10540925533894599])'
'Variable(name=e5, B=[[1 0 0]\n [0 1 0]\n [0 0 1]\n [1 1 1]\n [1 1 0]\n [0 1 1]], values=[0.9428090415820635, 0.1885618083164127, 0.09428090415820635])'
'Variable(name=e6, B=[[1 0 0]\n [0 1 0]\n [0 0 1]\n [1 1 1]], values=[0.7071067811865475, 0.1414213562373095, 0.07071067811865475])'
'Variable(name=o1, B=[[

In [16]:
varis['sys'] = variable.Variable( 'sys', np.eye( 3 ), ['fail', 'surv', 'unk'] )

cpm_sys_vname = copy.deepcopy( brs[0].names )
cpm_sys_vname.insert( 0, 'sys' )


cpms['sys'] = cpm.Cpm( variables=[varis[k] for k in cpm_sys_vname], no_child = 1, C = Csys, p = np.ones((len(Csys), 1), int) ) 

## Inference

### Case 1: No observation

In [17]:
var_elim_order_name = ['o'+str(i+1) for i in range(len(arcs))]+['e'+str(i+1) for i in range(len(arcs))] # observations first, components later
print(var_elim_order_name)
var_elim_order = [varis[k] for k in var_elim_order_name]

Msys = cpm.variable_elim( [cpms[v] for v in varis.keys()], var_elim_order )
print(Msys)
print( 'Pf_sys: ', str( Msys.p[ st_br_to_cs['fail'] ] ) )

['o1', 'o2', 'o3', 'o4', 'o5', 'o6', 'e1', 'e2', 'e3', 'e4', 'e5', 'e6']
Cpm(variables=['sys'], no_child=1, C=[[0]
 [1]], p=[[0.1018]
 [0.8982]]
Pf_sys:  [0.1018]


### Case 2: Observation

In [18]:
cnd_vars = ['o'+str(i+1) for i in range(len(arcs))]
cnd_states = [1, 1, 0, 1, 0, 1]

Mobs = cpm.condition( [cpms[v] for v in varis.keys()], cnd_vars, cnd_states )
Msys_obs = cpm.variable_elim( Mobs, var_elim_order )

print(Msys_obs)
print( 'Pf_sys: ', str( Msys_obs.p[ st_br_to_cs['fail'] ] / np.sum(Msys_obs.p) ) )

Cpm(variables=['sys'], no_child=1, C=[[0]
 [1]], p=[[2.76463167e-05]
 [5.51523633e-05]]
Pf_sys:  [0.33389804]
